# Step-by-Step Guide on hosting your basemap service on your own local server
<a href="https://leafletjs.com/" target="blank">LeafletJS</a>, <a href="https://www.mapbox.com/" target="blank">MapBox</a>, <a href="https://www.esri.com/" target="blank">Esri</a> etc. These are just a few of the map plugins which analysts, developers etc. who require various Geospatial visualisations tend to use. Needless to say, all these plugins have something in common - they require a basemap service for meaningful interactivity. Thanks to many open-sourced map services such as <a href="https://www.openstreetmap.org/" target="blank">OpenStreetMap(OSM)</a>, it is almost effortless to parse in its map service url for rendering:<br>
<u>Simple example of initialising a basemap using LeafletJS</u>
<pre>
    L.tileLayer('https://tile.openstreetmap.org/{z}/{x}/{y}.png', {
        attribution: 'Map data &copy; <a href="https://www.openstreetmap.org/">OpenStreetMap</a> contributors, <a href="https://creativecommons.org/licenses/by-sa/2.0/">CC-BY-SA</a>, Imagery © <a href="https://www.mapbox.com/">Mapbox</a>'
    }).addTo(mymap);
</pre>
---
However, this easy accessibility can backfire when an applicaiton fails to connect to the map service due to unforeseen circumstances such as - Absence of WiFi, Faulty Internet Connection, the Basemap service is no longer available etc. As a precaution, it is always wiser to consider hosting your own basemap locally on your local server especially on days when you are required to demonstrate or use your application at work.<br>
<u>Example of locally hosted basemap</u>
<img src="localhost_basemap_preview.png" height="150px" /><br>
Although there are several ways of doing it, my preferred choice is the make use of leaflet's mbtiles plugin known as <a href="https://gitlab.com/IvanSanchez/Leaflet.TileLayer.MBTiles" target="blank">Leaflet.TileLayer.MBTiles</a>, which requires some preparation steps I will be going through below. This includes a little bit of calculation, a local web server, the utility python package <a href="https://github.com/mapbox/mbutil " target="blank">mbutil</a> and the base leafletJS library(since the choice of mbtiles plugin is meant to be integrated with leafletJS).

## Step 1: Search for pre-existing basemap services online and pick your choice
First and foremost, decide on which basemap you are attempting to host offline so as to understand its tile url format (KIV: this is extremely important). A good place to start will be http://maps.stamen.com/, where there is a great variety of maps at no costs at all. Then, proceed to:
* Zoom to your desired minimum zoom level e.g. `11`, and ensure that your current browser viewport is currently capturing **ALL** the map tiles of the region/province/country you are targeting i.e. The bounding box should be within the browser. Example, assuming my objective is to host the basemap of Singapore, my browser shall look like this:<br>
<img src="sg_map_stamen.png" height="200px" />
* Proceed to right click and select "Inspect" or Ctrl+Shift+i (on Windows)
<img src="console_stamen_map.png" height="200px" /><br>
While different browsers have different configurations, the 2 essential components are the browser console and the `Elements` tab. As seen from the format of each map tile image rendered, it follows the conventional <a href="https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames" target="blank">Slippy map</a> format where the URL contains `{z}/{x}/{y}`,<br>
    - z: Zoom Level
    - x: X-Coordinate of Tile Image in pixels
    - y: Y-Coordinate of Tile Image in pixels
<br>    
FYI: The <b>{s}</b>-`{z}/{x}/{y}` is actually optional and refers to subdomain. More often than not due to heavy usage of these map services, subdomains `a`,`b`,`c` are configured in place. 
* Note that each tile image is rendered by a html image element `<img `. Hence, copy and paste the following JavaScript code snippet into the console:<br>
<pre>
    var imgs = document.getElementsByTagName("img");
    var coordinates = [];
    for(var i in imgs) {
        if(typeof imgs[i].classList !== "undefined") {
            if(imgs[i].classList.contains("leaflet-tile-loaded")) {
                var xy={
                    "x": parseInt(imgs[i].src.split("/")[5]),
                    "y": parseInt(imgs[i].src.split("/")[6].replace("@2x.png",""))
                };

                coordinates.push(xy);
            }
        }
    }
    console.log(JSON.stringify(coordinates));
</pre>
The output shall look similar to this:
<img src="javascript_output.png" height="200px" /><br>
This essential enables you to retrieve all the coordinates of the tile images at your desired minimum zoom level i.e. `11`<br>
<b>Explanation</b>: Since the format of each map tile rendered in http://maps.stamen.com/ follows the following format:<br>
`http://c.tile.stamen.com/toner/11/1614/1016@2x.png`, delimit it by the token `/` will transform this into an array:<br>

| 0 | 1 | 2 | 3 | 4 | 5 | 6 |
|---|---|---|---|---|---|---|
| http: |   | c.tile.stamen.com | toner | `11` | `1614` | `1016@2x.png` |

<br> The top column represents the index of the array. In which case, index of `x` can be found at position `5` of the array while position `y` can be found at position `6` of the array at zoom level 11 which is found at position `4` of the array. In case you have picked a different map service provider, change the index positions in the above code snippet accordingly to extract out the coordinates.<br>

## Step 2. Import python packages and Initialise functions for Latitude/Longitude or X/Y conversion

In [1]:
import math

The function `num2deg` transforms the `x` and `y` into latitude and longitude respectively based on zoom level (`zoom` parameter). Source: https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Lon..2Flat._to_tile_numbers_2

In [2]:
def num2deg(xtile, ytile, zoom):
  n = 2.0 ** zoom
  lon_deg = xtile / n * 360.0 - 180.0
  lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
  lat_deg = math.degrees(lat_rad)
  return (lat_deg, lon_deg)

The function `deg2num` transforms the `lat_deg` and `lon_deg` into x and y of a tile respectively based on zoom level (`zoom` parameter) Source: https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Lon..2Flat._to_tile_numbers_2

In [3]:
def deg2num(lat_deg, lon_deg, zoom):
  lat_rad = math.radians(lat_deg)
  n = 2.0 ** zoom
  xtile = int((lon_deg + 180.0) / 360.0 * n)
  ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
  return (xtile, ytile)

### 2.1 Transform all tile coordinates captured into pairs of latitude and longitude to render all coordinates as a GeoJSON:

Specify Minimum Zoom Level:

In [4]:
zoom=11

Initialise a GeoJSON Feature Collection object

In [5]:
geojsonObj={
  "type": "FeatureCollection",
  "features": []
}

Copy and Paste the output from Step 1 and store it into the variable `xy`:

In [6]:
xy=[
    {"x": 1614, "y": 1016}, {"x": 1615, "y": 1016}, {"x": 1614, "y": 1015}, {"x": 1615, "y": 1015}, 
    {"x": 1614, "y": 1017}, {"x": 1615, "y": 1017}, {"x": 1613, "y": 1016}, {"x": 1616, "y": 1016}, 
    {"x": 1613, "y": 1015}, {"x": 1616, "y": 1015}, {"x": 1613, "y": 1017}, {"x": 1616, "y": 1017}, 
    {"x": 1612, "y": 1016}, {"x": 1617, "y": 1016}, {"x": 1612, "y": 1015}, {"x": 1617, "y": 1015}, 
    {"x": 1612, "y": 1017}, {"x": 1617, "y": 1017}, {"x": 1615, "y": 1015}, {"x": 1616, "y": 1015}, 
    {"x": 1615, "y": 1016}, {"x": 1616, "y": 1016}, {"x": 1615, "y": 1016}, {"x": 1616, "y": 1016}, 
    {"x": 1615, "y": 1017}, {"x": 1616, "y": 1017}, {"x": 1614, "y": 1016}, {"x": 1615, "y": 1016}, 
    {"x": 1614, "y": 1015}, {"x": 1615, "y": 1015}, {"x": 1614, "y": 1017}, {"x": 1615, "y": 1017}, 
    {"x": 1613, "y": 1016}, {"x": 1616, "y": 1016}, {"x": 1613, "y": 1015}, {"x": 1616, "y": 1015}, 
    {"x": 1613, "y": 1017}, {"x": 1616, "y": 1017}, {"x": 1612, "y": 1016}, {"x": 1617, "y": 1016}, 
    {"x": 1612, "y": 1015}, {"x": 1617, "y": 1015}, {"x": 1612, "y": 1017}, {"x": 1617, "y": 1017}, 
    {"x": 1614, "y": 1018}, {"x": 1615, "y": 1018}, {"x": 1613, "y": 1018}, {"x": 1616, "y": 1018}, 
    {"x": 1612, "y": 1018}, {"x": 1617, "y": 1018}, {"x": 1615, "y": 1015}, {"x": 1616, "y": 1015}, 
    {"x": 1615, "y": 1016}, {"x": 1616, "y": 1016}, {"x": 1615, "y": 1017}, {"x": 1616, "y": 1017}, 
    {"x": 1615, "y": 1016}, {"x": 1616, "y": 1016}, {"x": 1615, "y": 1017}, {"x": 1616, "y": 1017}, 
    {"x": 1615, "y": 1018}, {"x": 1616, "y": 1018}
]

In [7]:
for item in xy:  
    feature={
      "type":"Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          num2deg(item["x"], item["y"], zoom)[1],
          num2deg(item["x"], item["y"], zoom)[0]
        ]
       },
        "properties": {
          "zoom": zoom
        } 
    }
    geojsonObj["features"].append(feature)

Output the GeoJSON into a file named: `output.geojson`

In [8]:
geojson=str(geojsonObj).replace("'","\"")

geojson_file = open("output.geojson", "w")
geojson_file.write(geojson)
geojson_file.close()

#### 2.2 Visualise map markers
Render the GeoJSON file onto any map rendering platform to view the coordinates on the actual map. Personally, http://geojson.io/ is a very convenient tool for temporal map visualisations. Using the above example, the coordinates transformed from the tiles at zoom level 11 shall look as such:
<img src="xy_map.png" height="200px" /><br>
Note that the dimensions of the above image shows 4 × 6 map markers, which is equivalent to 4 × 6 tile images.
## Step 3. Calculating the X,Y for other zoom levels
The tile coordinates of a slippy tiled basemap follows this equation:
* No. of Tiles= 2<sup>zoom</sup> × 2<sup>zoom</sup> E.g. Zoom Level 11 has 2<sup>11</sup> × 2<sup>11</sup> = 2048 × 2048
* Based on the markers visualised on the map, the number of tiles = 4 Tiles × 6 Tiles where:<br><br>
min(x)=1612<br>
max(x)=1616<br>
<br>
min(y)=1015<br>
max(y)=1018<br>
<pre>
|----------------
_ x: 1612..1616
|
|
|
|
y:  1015
    ..
    1018
</pre>
---
Hence, for zoom level 11+1=`12`, No. of Tiles = 2<sup>12</sup> × 2<sup>12</sup> = 4096 × 4096<br>
(4/2048)×4096 × (6/2048)×4096 tiles = 8 × 12 Tiles<br><br>
min(x)=1612/4 × 8 = 3224<br>
max(x)=1616/4 × 8 = 3232<br>
<br>
min(y)=1015/6 × 12 = 2030<br>
max(y)=1018/6 × 12 = 2036<br>
<pre>
|----------------
_ x: 3224..3232
|
|
|
|
y:  2030
    ..
    2036
</pre>

### 3.1 Proof of Concept that the above equation is valid

In [9]:
urlPrefix="http://c.tile.stamen.com/toner/"
tileUrls=[]
zoom_level=12
for x in range(3224,3232+1,1):
  for y in range(2030,2036+1,1):
      tileUrl=urlPrefix+str(zoom_level) + "/" + str(x) + "/" + str(y) + "@2x.png"
      tileUrls.append(tileUrl)

Proceed to output some of the tileUrls and select to view a tile image (if it is a broken link, the map service is probably down so seek another map service. Else, the image rendered does in fact belong to the basemap you are trying to render):

In [10]:
tileUrls[:12]

['http://c.tile.stamen.com/toner/12/3224/2030@2x.png',
 'http://c.tile.stamen.com/toner/12/3224/2031@2x.png',
 'http://c.tile.stamen.com/toner/12/3224/2032@2x.png',
 'http://c.tile.stamen.com/toner/12/3224/2033@2x.png',
 'http://c.tile.stamen.com/toner/12/3224/2034@2x.png',
 'http://c.tile.stamen.com/toner/12/3224/2035@2x.png',
 'http://c.tile.stamen.com/toner/12/3224/2036@2x.png',
 'http://c.tile.stamen.com/toner/12/3225/2030@2x.png',
 'http://c.tile.stamen.com/toner/12/3225/2031@2x.png',
 'http://c.tile.stamen.com/toner/12/3225/2032@2x.png',
 'http://c.tile.stamen.com/toner/12/3225/2033@2x.png',
 'http://c.tile.stamen.com/toner/12/3225/2034@2x.png']

At this stage, since we know how to calculate and retrieve all the basemap tile images we require, we should set up a folder structure in accordance to Slippy map rules to save the respective images.<br><br>
This is the browser backend from the site hosting the map service.<br>
<img src="basemap_folder_structure.png" width="250px" /><br>
We can infer from the above that map tile images follow the following folder structure:
<pre>
    |--zoom level
    |     |---x-tile coordinate
    |           |---y-tile coordinate
</pre>
As such, we shall proceed to run the following cell to create the required folder structure. However before doing so, decide on the zoom upper limit of your basemap. Conventionally, `17` or `18` are good choices as it enables users to view street level details. For demonstration sake I shall set my upper limit to just zoom level `15`.

### 3.2 Creation of Folder Structure

Recall that at zoom level 11,<br>
Dimensions are: 4 × 6 Tiles<br>
<br>
min(x)=1612<br>
max(x)=1616<br>
<br>
min(y)=1015<br>
max(y)=1018<br>
<pre>
|----------------
_ x: 1612..1616
|
|
|
|
y:  1015
  ..
  1018
</pre>
---
So at zoom level `n`, since no. of Tiles= 2<sup>zoom</sup> × 2<sup>zoom</sup>,<br>
Dimensions shall be: 4(2<sup>n</sup>) × 6(2<sup>n</sup>) Tiles<br>
<br>
min(x)=1612/4 × 4(2<sup>n</sup>)<br>
max(x)=1616/4 × 4(2<sup>n</sup>)<br>
<br>
min(y)=1015/6 × 6(2<sup>n</sup>)<br>
max(y)=1018/6 × 6(2<sup>n</sup>)<br>
<pre>
|----------------
_ x: min(x)..max(x)
|
|
|
|
y:  min(y)
  ..
  max(y)
</pre>
Based on the above logic, a function can be created in order to stream the coordinates of the tile images at any zoom level.

In [11]:
from math import pow
import os

minZoomLevel=11 # IMPORTANT! Ensure that your minZoomLevel is the same as the one you decided intially
maxZoomLevel=15

directoryPrefix="toner/" # assume it is being saved in a folder named toner

def initDirectoryStructure(n): # where n refers to the zoom level
    minX=(1612/4)*4*pow(2,n-minZoomLevel)
    maxX=(1616/4)*4*pow(2,n-minZoomLevel)
    
    minY=(1015/6)*6*pow(2,n-minZoomLevel)
    maxY=(1018/6)*6*pow(2,n-minZoomLevel)
    
    minX=int(minX)
    maxX=int(maxX)
    minY=int(minY)
    maxY=int(maxY)
    
    for x in range(minX,maxX+1,1):
      for y in range(minY,maxY+1,1):
            directory=directoryPrefix+str(n) + "/" + str(x)
            if not os.path.exists(directory):
                os.makedirs(directory)

for z in range(minZoomLevel,maxZoomLevel+1,1):
    initDirectoryStructure(z)

## Step 4. Stream the tile images into the local folders you have just 
After initialisation of the folder structure, we can now retrieve tile images of zoom levels `11` to `18` (upper limit of basemap zoom) from the basemap service.
### Pause and take note: This step can take up to a few hours if the no. of tile images are huge. Generally my basemaps range from zoom levels `11` to `17` and the entire folder takes up approximately ~500MB. For Satellite maps however, it often amounts to a few GB so ensure you have enough disk space before running the next cell.

In [12]:
import requests 

basemapUrlPrefix="http://c.tile.stamen.com/toner/" # the subdomain is c in the url. can be a or b too.
basemapUrlSuffix="@2x.png"

def streamTileImages(n): # where n refers to the zoom level
    minX=(1612/4)*4*pow(2,n-minZoomLevel)
    maxX=(1616/4)*4*pow(2,n-minZoomLevel)
    
    minY=(1015/6)*6*pow(2,n-minZoomLevel)
    maxY=(1018/6)*6*pow(2,n-minZoomLevel)
    
    minX=int(minX)
    maxX=int(maxX)
    minY=int(minY)
    maxY=int(maxY)
    
    for x in range(minX,maxX+1,1):
      for y in range(minY,maxY+1,1):
            # send a HTTP request to retrieve the tile image file
            basemapTileUrl=basemapUrlPrefix+str(n) + "/" + str(x) + "/" + str(y) + basemapUrlSuffix
            r = requests.get(basemapTileUrl)
            # proceed to save it as a local image file in the folders created in step 3
            save_as_filename=directoryPrefix+str(n) + "/" + str(x) + "/" + str(y) + ".png"
            with open(save_as_filename,"wb") as local_tile_image:
                local_tile_image.write(r.content) 

for z in range(minZoomLevel,maxZoomLevel+1,1):
    streamTileImages(z)

#### Here is a summary of everything which has been output from the above steps:<br>
<img src="streamed_tile_images.png" height="200px" />

### 4.1 Host Folders on a web server to ensure tile images are saved successfully
Finally, to ensure that the correct tile images were saved, a web server should be set up and call the basemap directly from the folders (packaging can be done after ensuring the basemap renders correctly).<br>
For demonstration sake, I shall just use `Flask` and `werkzeug` to set up a web server directly in this notebook to host the basemap.<br>
Note: Include the leaflet library, <a href="https://unpkg.com/leaflet@1.0.0/dist/leaflet.css" target="blank">leaflet stylesheet</a>, <a href="https://unpkg.com/leaflet@1.0.0/dist/leaflet-src.js" target="blank">base leafletJS</a>, < href="http://ivansanchez.gitlab.io/Leaflet.TileLayer.MBTiles/Leaflet.TileLayer.MBTiles.js" target="blank">L.TileLayer.mbTiles plugin</a> and the plugin's dependency, <a href="https://unpkg.com/sql.js@0.3.2/js/sql.js" target="blank">sql.js</a> into the web application and proceed to include the following code snippet between the `<script></script>` tags to initialise your basemap:<br>
`<script>`
<pre>
    var map = new L.Map("map");
    var basemapLayer=L.tileLayer("static/maps/toner/{z}/{x}/{y}.png", {
      minZoom: 11,
      maxZoom: 15
    }).addTo(map);

    var xhr = new XMLHttpRequest();
    xhr.open("GET", "static/output.geojson");
    xhr.setRequestHeader("Content-Type", "application/json");
    xhr.responseType = "json";
    xhr.onload = function() {
        if (xhr.status !== 200) {
          return
        }
        var geoJSONLayer=L.geoJSON(xhr.response);
        map.fitBounds(geoJSONLayer.getBounds())
    };
    xhr.send();
</pre> 
`</script>`<br>

---
<b>Explanation</b>: 
* The above code snippet intialises a new map object with leafletJS. 
* The file `output.geojson` generated from Step 2 is called via AJAX and proceeds to set the map view is within the basemap's bbox.

In [13]:
from flask import Flask
from flask import render_template
from werkzeug.wrappers import Request, Response

In [14]:
app = Flask(__name__)

@app.route("/")
def index():
    title="Local Basemap"
    return render_template("index.html", message=title, template_folder="templates", static_folder="static")

if __name__ == "__main__":
    from werkzeug.serving import run_simple
    run_simple("localhost", 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Oct/2020 23:06:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:06:10] "GET /static/css/leaflet.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:06:10] "GET /static/js/leaflet.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:06:10] "GET /static/js/sql.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:06:10] "GET /static/js/Leaflet.TileLayer.MBTiles.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:06:10] "GET /static/loading.gif HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:06:11] "GET /static/output.geojson HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:06:11] "GET /static/maps/toner/11/1614/1015.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:06:11] "GET /static/maps/toner/11/1613/1016.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:06:11] "GET /static/maps/toner/11/1615/1016.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:06:11] "GET /static/maps/toner/11/1614/1017.png HTTP/1.1" 200 -
127.0.0.1

127.0.0.1 - - [28/Oct/2020 23:07:11] "GET /static/maps/toner/15/25822/16270.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:11] "GET /static/maps/toner/15/25826/16270.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:11] "GET /static/maps/toner/15/25822/16268.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:11] "GET /static/maps/toner/15/25826/16268.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:12] "GET /static/maps/toner/15/25822/16271.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:12] "GET /static/maps/toner/15/25826/16271.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:12] "GET /static/maps/toner/15/25824/16267.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:12] "GET /static/maps/toner/15/25825/16267.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:12] "GET /static/maps/toner/15/25823/16267.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:12] "GET /static/maps/toner/15/25826/16267.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:12] "GE

127.0.0.1 - - [28/Oct/2020 23:07:31] "GET /static/maps/toner/13/6456/4067.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:31] "GET /static/maps/toner/13/6459/4068.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:31] "GET /static/maps/toner/13/6459/4066.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:31] "GET /static/maps/toner/13/6455/4068.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:31] "GET /static/maps/toner/13/6454/4067.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:31] "GET /static/maps/toner/13/6456/4068.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:31] "GET /static/maps/toner/13/6454/4066.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:31] "GET /static/maps/toner/13/6459/4067.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:31] "GET /static/maps/toner/13/6456/4066.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:31] "GET /static/maps/toner/13/6458/4067.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 23:07:31] "GET /static/maps/toner

Example of initial Basemap View:<br>
<img src="initial_basemap_view.png" height="200px" /><br>
Zoomed-In View:<br>
<img src="zoomed_in_basemap_view.png" height="200px" /><br>
#### Congratulations! You now have your basemap rendered on your local server.¶ 
### 4.2 Package the folder with `mbutil`
Now that the basemap is functional, the folder can be packaged into a single mbTile file. THis is done via the python package <a href="https://pypi.org/project/mbutil/" target="blank">mbutil</a>. Proceed to run the next cell to install the package.